<a href="https://colab.research.google.com/github/Imteaz1998/Genrative-AI/blob/main/13_Custom_ChatBot_for_Any_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers

!pip install tiktoken

In [2]:
pip install --upgrade google-generativeai

In [3]:
!pip install tokenizers
!pip install faiss-cpu
!pip -q install unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.2 MB/s eta 0:00:00


In [5]:
!pip install numpy
!pip install nltk

In [7]:
pip install -qU "langchain[google-vertexai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 93.1 MB/s eta 0:00:00


In [24]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from langchain.chat_models import init_chat_model
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
os.environ['GEMINI_API_KEY'] = "API"

In [ ]:
os.environ['HuggingFace_API_KEY'] = "API"

In [11]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [12]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [13]:
data

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their ap

In [14]:
len(data)

4

Split text in chunks

In [15]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [16]:
text_chunks=text_splitter.split_documents(data)

In [17]:
len(text_chunks)

86

In [18]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

In [19]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-19-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-19-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

768

#**Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [21]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [26]:
from google.colab import userdata
Google_sec_key=userdata.get("GOOGLEAPI")

In [29]:
from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory
llm = GoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=Google_sec_key,
    temperature=0.5,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [30]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

"Harry Potter is a series of seven fantasy novels by J.K. Rowling that chronicles the life of a young wizard, Harry Potter, who discovers on his eleventh birthday that he is a wizard and destined to attend Hogwarts School of Witchcraft and Wizardry. Throughout the series, Harry, with his friends Ron Weasley and Hermione Granger, battles the dark wizard Lord Voldemort, who murdered Harry's parents and seeks to conquer the wizarding world. The books explore themes of love, friendship, prejudice, good versus evil, and the power of choice. Ultimately, Harry triumphs over Voldemort, restoring peace and balance to the wizarding world."

#Initialize the Retrieval QA with Source Chain

In [31]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [32]:
result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)

<ipython-input-32-6dc5d9fb268a>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)


In [33]:
result['answer']

"Vicuna-13B, after being fine-tuned with 70K user-shared ChatGPT conversations, can generate more detailed and well-structured answers compared to Alpaca, with the quality being on par with ChatGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90% the quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90% of cases. In 45% of the questions, GPT-4 rates Vicuna's response as better or equal to ChatGPT's. Vicuna’s total score is 92% of ChatGPT’s. However, Vicuna is not good at tasks involving reasoning or mathematics, and it may have limitations in accurately identifying itself or ensuring the factual accuracy of its outputs.\n"

In [34]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [35]:
wrapped_text

"Vicuna-13B, after being fine-tuned with 70K user-shared ChatGPT conversations, can generate more detailed and well-structured answers compared to Alpaca, with the quality being on par with ChatGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90% the quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90% of cases. In 45% of the questions, GPT-4 rates Vicuna's response as better or equal to\nChatGPT's. Vicuna’s total score is 92% of ChatGPT’s. However, Vicuna is not good at tasks involving reasoning or mathematics, and it may have limitations in accurately identifying itself or ensuring the factual accuracy of its outputs."

In [36]:
result=chain({"question": "What is is stableLM?"}, return_only_outputs=True)

In [37]:
result['answer']

'StableLM is a new open source language model released by Stability AI.\n'

In [38]:
while True:
  query=input(f"Prompt: ")
  if query == 'exit':
    print('Exiting')
    sys.exit()
  if query =='':
    continue
  result=chain({'question':query})
  print(f"Answer: " +result["answer"])

Prompt: Can you please share some details about MPT-7b Model
Answer: The MPT-7B model was trained on the MosaicML platform using A100-40GB and A100-80GB GPUs from Oracle Cloud. The training of the base MPT-7B model took approximately 9.5 days on 440xA100-40GB GPUs and cost around $200k. The MosaicML platform detected and addressed 4 hardware failures and resumed the training run automatically. When using `model.generate(...)`, the out-of-the-box performance of MPT-7B is 1.5x-2x faster than other 7B models like LLaMa-7B.


Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
